In [12]:
from src.blackscholes import *
from src.pinn import *
from src.collocation import *
from src.differential import BlackScholesDifferential
from src.differential import SchrodingerDifferential
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
from datetime import datetime
from scipy.interpolate import griddata
from pathlib import Path

# ---------- CONFIG ----------
# Use current working directory
DATA_DIR = Path.cwd()
OPT_TRAIN_FILE = DATA_DIR / 'OPT_RELIANCE_20250529.csv'
OPT_TEST_FILE  = DATA_DIR / 'OPT_RELIANCE_20250626.csv'
EQT_TRAIN_FILE = DATA_DIR / 'EQT_RELIANCE_20250529.csv'
EQT_TEST_FILE  = DATA_DIR / 'EQT_RELIANCE_20250626.csv'
OUTPUT_MAT_FILE = DATA_DIR / 'PINN_TRAIN_DATA.mat'

# ---------- HELPERS ----------
def load_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Timestamp'], dayfirst=True)
    df['logS'] = np.log(df['LTP'].replace(0, np.nan)).fillna(method='ffill')
    df['TimeToMaturity'] = (pd.to_datetime(df['ExpiryDate'], dayfirst=True) - df['Timestamp']).dt.total_seconds()
    df['TimeToMaturity'] = df['TimeToMaturity'] / df['TimeToMaturity'].max()  # Normalize [0,1]
    df['MidPrice'] = (df['BuyPrice'] + df['SellPrice']) / 2
    df['Spread'] = df['SellPrice'] - df['BuyPrice']
    df['Imbalance'] = (df['BuyQty'] - df['SellQty']) / (df['BuyQty'] + df['SellQty'] + 1e-5)
    df['EntropicVol'] = df['LTP'].pct_change().rolling(5).std().fillna(0)
    df['LogReturn'] = np.log(df['LTP'] / df['LTP'].shift(1)).fillna(0)
    df['Entropy'] = (df['LogReturn']**2).rolling(5).mean().fillna(0)
    df['RegimeFlag'] = ((df['LogReturn'].abs() > 0.01) | (df['Entropy'] > 0.0005)).astype(int)  # volatility/gap flag
    df['Delta'] = 0.5  # Placeholder for simplicity
    df['IV'] = df['Spread'] / (df['LTP'] + 1e-5)  # Crude IV proxy

    return df

# ---------- MAIN ----------
def main():
    opt_df = load_data(OPT_TRAIN_FILE)

    x = opt_df['logS'].values
    t = opt_df['TimeToMaturity'].values
    re_psi = opt_df['LTP'].values
    im_psi = opt_df['Imbalance'].values * opt_df['Spread'].values

    uu = re_psi + 1j * im_psi

    # Engineered features
    feature_dict = {
        'IV': opt_df['IV'].values,
        'Delta': opt_df['Delta'].values,
        'OI': opt_df['OpenInterest'].values,
        'Entropy': opt_df['Entropy'].values,
        'RegimeFlag': opt_df['RegimeFlag'].values
    }

    # Create mesh grid for interpolation
    x_grid = np.linspace(np.min(x), np.max(x), 256)
    t_grid = np.linspace(np.min(t), np.max(t), 100)
    x_mesh, t_mesh = np.meshgrid(x_grid, t_grid)
    uu_grid = griddata((x, t), uu, (x_mesh, t_mesh), method='linear', fill_value=0)

    # Interpolate features
    feature_grids = {}
    for name, feat in feature_dict.items():
        feature_grids[name] = griddata((x, t), feat, (x_mesh, t_mesh), method='linear', fill_value=0)

    # Final output dict
    output = {
        'x': x_grid,
        'tt': t_grid,
        'uu': uu_grid,
    }
    for name, grid in feature_grids.items():
        output[name] = grid

    # Save to .mat
    sio.savemat(OUTPUT_MAT_FILE, output)
    print(f" Saved MAT file: {OUTPUT_MAT_FILE}")

if __name__ == '__main__':
    main()


/tmp/ipykernel_44994/437918939.py:20: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df = pd.read_csv(file_path, parse_dates=['Timestamp'], dayfirst=True)
/tmp/ipykernel_44994/437918939.py:21: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['logS'] = np.log(df['LTP'].replace(0, np.nan)).fillna(method='ffill')
/tmp/ipykernel_44994/437918939.py:22: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df['TimeToMaturity'] = (pd.to_datetime(df['ExpiryDate'], dayfirst=True) - df['Timestamp']).dt.total_seconds()


 Saved MAT file: /mnt/c/Meril/Python/1D_BS_SE_pinn-black-scholes-main/PINN_TRAIN_DATA.mat


In [5]:
import pandas as pd
import numpy as np
import scipy.io as sio
from pathlib import Path
from scipy.interpolate import griddata

DATA_DIR = Path.cwd()
EQT_TRAIN_FILE = DATA_DIR / 'EQT_RELIANCE_20250529.csv'
EQT_OUTPUT_MAT_FILE = DATA_DIR / 'EQT_PINN_TRAIN_DATA.mat'

def load_equity_data(file_path):
    df = pd.read_csv(file_path, parse_dates=['Timestamp'], dayfirst=True)
    df['logS'] = np.log(df['LTP'].replace(0, np.nan)).fillna(method='ffill')
    df['Time'] = (df['Timestamp'] - df['Timestamp'].min()).dt.total_seconds()
    df['Time'] = df['Time'] / df['Time'].max()

    df['Return'] = np.log(df['LTP'] / df['LTP'].shift(1)).fillna(0)
    df['Entropy'] = (df['Return']**2).rolling(5).mean().fillna(0)
    df['Volume'] = df['LTQ'].fillna(0)
    df['RegimeFlag'] = ((df['Return'].abs() > 0.01) | (df['Entropy'] > 0.0005)).astype(int)

    # --- Enhanced Im(ψ): Signed smoothed entropy ---
    signed_entropy = np.sign(df['Return']) * df['Entropy']
    im_psi = signed_entropy.ewm(span=10, adjust=False).mean()
    im_psi = np.tanh(im_psi / (im_psi.std() + 1e-8))  # bounded for smoothness

    df['ImPsi'] = im_psi
    return df

def create_equity_mat():
    eqt_df = load_equity_data(EQT_TRAIN_FILE)

    x = eqt_df['logS'].values
    t = eqt_df['Time'].values
    re_psi = eqt_df['LTP'].values
    im_psi = eqt_df['ImPsi'].values
    uu = re_psi + 1j * im_psi

    feature_dict = {
        'Entropy': eqt_df['Entropy'].values,
        'Volume': eqt_df['Volume'].values,
        'RegimeFlag': eqt_df['RegimeFlag'].values,
    }

    x_grid = np.linspace(np.min(x), np.max(x), 256)
    t_grid = np.linspace(np.min(t), np.max(t), 100)
    x_mesh, t_mesh = np.meshgrid(x_grid, t_grid)
    uu_grid = griddata((x, t), uu, (x_mesh, t_mesh), method='linear', fill_value=0)

    feature_grids = {
        name: griddata((x, t), val, (x_mesh, t_mesh), method='linear', fill_value=0)
        for name, val in feature_dict.items()
    }

    output = {
        'x': x_grid,
        'tt': t_grid,
        'uu': uu_grid,
    }
    output.update(feature_grids)

    sio.savemat(EQT_OUTPUT_MAT_FILE, output)
    print(f".mat file saved: {EQT_OUTPUT_MAT_FILE}")
    return EQT_OUTPUT_MAT_FILE.name, list(output.keys())

# Run
if __name__ == "__main__":
    create_equity_mat()


/tmp/ipykernel_44994/1172055242.py:12: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df = pd.read_csv(file_path, parse_dates=['Timestamp'], dayfirst=True)
/tmp/ipykernel_44994/1172055242.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['logS'] = np.log(df['LTP'].replace(0, np.nan)).fillna(method='ffill')


.mat file saved: /mnt/c/Meril/Python/1D_BS_SE_pinn-black-scholes-main/EQT_PINN_TRAIN_DATA.mat


In [ ]:
import pandas as pd
import numpy as np
from scipy.io import savemat
from scipy.stats import norm
from scipy.interpolate import griddata, SmoothBivariateSpline
import os

# === CONFIGURATION ===
OPTION_CSV = "OPT_RELIANCE_20250626.csv"
EXPIRY = "20250626"
SYMBOL = "RELIANCE"
KERNEL_SMOOTH = 3  # for smoothing returns
GAP_THRESHOLD = 5
ENTROPY_WINDOW = 15

# === BLACK-SCHOLES HELPER ===
def bs_delta(S, K, T, r, sigma, option_type='C'):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    if option_type == 'C':
        return norm.cdf(d1)
    else:
        return -norm.cdf(-d1)

def bs_iv_call(S, K, T, r, market_price, tol=1e-5, max_iter=100):
    from scipy.optimize import brentq
    def bs_price(sigma):
        d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
        d2 = d1 - sigma*np.sqrt(T)
        return S * norm.cdf(d1) - K * np.exp(-r*T) * norm.cdf(d2)
    try:
        return brentq(lambda sigma: bs_price(sigma) - market_price, 1e-6, 5.0, maxiter=max_iter)
    except Exception:
        return np.nan

# === LOAD & PREPROCESS DATA ===
df = pd.read_csv(OPTION_CSV)
df.dropna(subset=['LTP', 'BuyPrice', 'OpenInterest', 'StrikePrice', 'DTE'], inplace=True)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['MidPrice'] = 0.5 * (df['BuyPrice'] + df['SellPrice'])

# === GRID CREATION ===
x_vals = np.sort(df['StrikePrice'].unique())
tt_vals = np.sort(df['DTE'].unique()) / df['DTE'].max()
Nx, Nt = len(x_vals), len(tt_vals)

# === SURFACE CONSTRUCTION ===
grid_df = df.copy()
grid_df['x'] = grid_df['StrikePrice']
grid_df['t'] = grid_df['DTE'] / df['DTE'].max()
grid_df['log_moneyness'] = np.log(df['Underlying'].astype(float) / df['StrikePrice'].astype(float))
pivot_ltp = grid_df.pivot(index='DTE', columns='StrikePrice', values='LTP').fillna(0)
pivot_mid = grid_df.pivot(index='DTE', columns='StrikePrice', values='MidPrice').fillna(0)
uu_real = pivot_ltp.values
uu_imag = pivot_ltp.pct_change().rolling(KERNEL_SMOOTH, axis=0).mean().fillna(0).values

# === MASK ===
mask = (uu_real > 0).astype(int)

# === ENGINEERED FEATURES ===
oi = grid_df.pivot(index='DTE', columns='StrikePrice', values='OpenInterest').fillna(0).values
entropy = np.log(1 + np.abs(pivot_ltp.pct_change().rolling(ENTROPY_WINDOW, axis=0).std())).fillna(0).values
vol_shock = (entropy > np.nanpercentile(entropy, 85)).astype(int)
gap_open = (pivot_ltp - pivot_mid > GAP_THRESHOLD).astype(int)
event_day = grid_df.groupby('DTE')['Timestamp'].transform(lambda x: x.duplicated()).values.reshape(Nt, Nx).astype(int)

# === DELTA & IV ===
r = 0.05
delta = np.zeros((Nt, Nx))
iv = np.zeros((Nt, Nx))
for i, t in enumerate(np.sort(df['DTE'].unique())):
    for j, k in enumerate(x_vals):
        row = df[(df['DTE'] == t) & (df['StrikePrice'] == k)]
        if not row.empty:
            S = row['Underlying'].values[0]
            T = t / 365
            try:
                market_price = row['LTP'].values[0]
                iv[i, j] = bs_iv_call(S, k, T, r, market_price)
                delta[i, j] = bs_delta(S, k, T, r, iv[i, j])
            except Exception:
                iv[i, j] = np.nan
                delta[i, j] = np.nan
iv = np.nan_to_num(iv, nan=0.0)
delta = np.nan_to_num(delta, nan=0.0)

# === FEATURES TENSOR ===
features = np.stack([oi, entropy, vol_shock, gap_open, event_day, iv, delta], axis=2)

# === SAVE TO .mat ===
savemat(f'PINN_TRAIN_DATA_{EXPIRY}.mat', {
    'x': x_vals,
    'tt': tt_vals,
    'uu': uu_real + 1j * uu_imag,
    'features': features,
    'mask': mask,
    'symbol': SYMBOL,
    'expiry': EXPIRY
})
